In [1]:
import numpy as np

In [ ]:
#ratio of customers
alpha = np.array(np.random.dirichlet((1,2,3,4,5,6), 1))
alpha_0 = alpha[0,0] #ratio of customers landing on another website (competitor)
alpha_i = alpha[0,1:] #vector with ratio of customers landing on the webpage in which product Pi is primary

In [ ]:
#1) Creation of the graph, influence click probabilities between products
#All the edges connected
graph = np.zeros((5,5))
lambda_ = 0.5 #probability to see the secondary product in the second slot

for i in range(5):
  j = np.random.choice([x for x in range(5) if x != i ],2, replace = False) #secondary slots indexes (0,1,2,3,4,5)-{i=primary}
  graph[i,j[0]] = np.random.uniform(0.0, 1, 1)
  graph[i,j[1]] = np.random.uniform(0.0, 1, 1) * lambda_

print(graph)

#2) Not all the edges are connected
graph_2 = np.zeros((5,5))
lambda_ = 0.5

for i in range(5):
  k = np.random.choice(3,1) # k = 0,1,2
  j = np.random.choice([x for x in range(5) if x != i ],k) #secondary slots indexes
  if k ==2:
    graph_2[i,j[0]] = np.random.uniform(0.0, 1, 1)
    graph_2[i,j[1]] = np.random.uniform(0.0, 1, 1) * lambda_
  elif k ==1:
    graph_2[i,j] = np.random.uniform(0.0, 1, 1)
  #if k==0 probabilities = 0

print("***********************************************************")
print(graph_2)

In [ ]:
#Product
class Product:
  def __init__(self, prices, margins, conversion_probabilities):
    self.prices = prices #vector length 4
    self.margins = margins #vector length 4
    self.conversion_prob = conversion_probabilities #vector of random variables whose means are unknown

#For Ecommerce class
def ProductsGenerator(num_products, Price_matrix, Margin_matrix, Conv_matrix):
  products = np.zeros(num_products)
  for i in range(num_products):
    products[i] = Product(Price_matrix[i,:], Margin_matrix[i,:], Conv_matrix[i,:])


In [ ]:
#User
#Each user has a reservation price and the index of the primary product (of the alpha_i[primary_product] population)
class User:
  def __init__(self, reservation_price, primary_product):
    self.rp = reservation_price #vector length 5 (one for each product)
    self.primary_product = primary_product #index of the primary product

class User0(User):
  def __init__(self, reservation_price = np.random.choice(10,1), primary_product = np.random.choice(5,1)):#to improve: reservation price, primary product
    User.__init__(self, reservation_price, primary_product) #different random distribution to create reservation price)

class User1(User):
  def __init__(self, reservation_price = np.random.choice(10,1), primary_product = np.random.choice(5,1)):#to improve reservation price, primary product, different from class 0
    User.__init__(self, reservation_price, primary_product) #different random distribution to create reservation price)

class User2(User):
  def __init__(self, reservation_price = np.random.choice(10,1), primary_product = np.random.choice(5,1)):#to improve reservation price, primary product
    User.__init__(self, reservation_price, primary_product) #different random distribution to create reservation price)

In [ ]:
def whichUser(binary_vector, primary_product):
    if np.sum(binary_vector==0):
      return User0(primary_product)
    elif np.sum(binary_vector==1):
        return User1(primary_product)
    elif np.sum(binary_vector==2):
      return User2(primary_product)
      
#each day contains a vector of users: creating a day, we create a vector of users suddivided in three classes, based on the binary_vector.
class Day: 
  def __init__(self,num_users, alpha_i, binary_vector):
    daily_users = []
    for i in range(alpha_i):
      for j in range(alpha_i[i]*num_users):
        self.daily_users.append(whichUser(binary_vector, i))#i index of the alpha ratio -> index of primary_product

In [ ]:
class Ecommerce:
  def __init__(self,num_products, num_days):
    self.num_products = num_products
    self.products = []
    self.num_days = num_days
    self.Days = [] #each day contains a vector of users

  def ProductsGenerator(self,num_products, Price_matrix, Margin_matrix, Conv_matrix):
    for i in range(num_products):
      self.products = self.products.append(Product(Price_matrix[i,:], Margin_matrix[i,:], Conv_matrix[i,:]))
    
  def whichUser(binary_vector, primary_product):
    if np.sum(binary_vector==0):
      return User0(primary_product)
    elif np.sum(binary_vector==1):
        return User1(primary_product)
    elif np.sum(binary_vector==2):
      return User2(primary_product)

    
  def GenerateDay(self, num_users, alpha_i, binary_vector):
    for i in range(self.num_days):
      self.Days.append(Day(num_users, alpha_i, binary_vector))

In [4]:
#simulating a click history of an user
#if a primary product is shown again, the click probability is 0, so its node can't be activated
#for every primary product, the pair and the order of the secondary products to display is fixed by the business unit and cannot be controlled

def simulate_purchase(init_prob_matrix, n_steps_max, primary_product):#index of primary product
    prob_matrix = init_prob_matrix.copy()#graph matrix
    n_nodes = prob_matrix.shape[0]
    active_nodes = np.zeros(n_nodes)
    active_nodes[primary_product] = 1 #the seed is the user's primary product shown
    history = np.array([active_nodes])
    newly_active_nodes = active_nodes
    old_active_nodes = np.array([active_nodes])#history of past clicks (nodes)
    t = 0
    while(t<n_steps_max and np.sum(newly_active_nodes)>0):
    #users click on a secondary product only if they bought the primary (price<user's reservation price)
    #just one product has to be with a price smaller than user's reservation price, not all. 
        p = (prob_matrix.T * active_nodes).T
        #find active edges simulating if the influence probabilities are higher than user's wish to click
        activated_edges = p > np.random.rand(p.shape[0], p.shape[1]) #user's random possibility to click
        prob_matrix = prob_matrix * ((p!=0)==activated_edges)
        newly_active_nodes = (np.sum(activated_edges, axis=0)>0) * (1-active_nodes)

        for i in range(len(old_active_nodes)):#to remove clicks on products already seen
          newly_active_nodes[old_active_nodes[i].reshape(-1)!=0] = 0 #no possibility to click again on a product already shown as primary
          #newly_active_nodes[user.reservation_price[active_nodes==1]>Price_vector[active_nodes==1]] = 0  #price_vector holds P1,P2... chosen prices

        active_nodes = newly_active_nodes
        old_active_nodes = np.array([old_active_nodes,active_nodes])
        history = np.concatenate((history, [newly_active_nodes]),axis=0)
        t+=1
    print(history)
    return history

#To fix and improve
#given a primary product (seed) estimate probabilities to click on the other products
#estimate probabilities to click on a product, based on old click histories of users
def estimate_probabilities(dataset, node_index, n_nodes):
    estimated_prob = np.ones(n_nodes)*1.0/(n_nodes-1)
    credits = np.zeros(n_nodes)
    occur_v_active = np.zeros(n_nodes)
    #n_episodes = len(dataset)
    for episode in dataset:
        idx_w_active = np.argwhere(episode[:, node_index] ==1).reshape(-1)
        #idx_w_active = np.argwhere(episode[node_index,:] ==1).reshape(-1)
        if len(idx_w_active)>0 and np.all(idx_w_active>0):
            active_nodes_in_prev_step = episode[idx_w_active - 1, :].reshape(-1)
            credits += (active_nodes_in_prev_step)/np.sum(active_nodes_in_prev_step)
        for v in range(0,n_nodes):
            if(v!=node_index):
                idx_v_active = np.argwhere(episode[:, v] == 1).reshape(-1)
                if len(idx_v_active)>0 and (np.all(idx_v_active<idx_w_active) or len(idx_v_active)>0):
                    occur_v_active[v]+=1
    estimated_prob = np.nan_to_num(credits/occur_v_active)
    return estimated_prob

In [ ]:
#Testing
graph = np.zeros((5,5))
lambda_ = 0.5
for i in range(5):
  j = np.random.choice([x for x in range(5) if x != i ],2, replace=False) #secondary slots indexes
  graph[i,j[0]] = np.random.uniform(0.0, 1, 1)
  graph[i,j[1]] = np.random.uniform(0.0, 1, 1)

num_products = 5
n_nodes = num_products

num_users = 1 #each day
n_episodes = num_users

node_index = 2#User.primary_product #the seed is the primary product of the alpha_i ratio
dataset = []

prob_matrix = graph #probability edges

#generate a dataset with n=n_episodes possible chain of activated_nodes (paths activated)
#each episode is a user landing on the webpage
for e in range(0,n_episodes):#we have to iterate on each user and to specify his class and his primary_product (alpha_i class)
    dataset.append(simulate_purchase(init_prob_matrix = prob_matrix, n_steps_max=5, primary_product=2))#max 5 steps because there can't be repetitions

#estimated_prob = estimate_probabilities(dataset=dataset, node_index=node_index, n_nodes=n_nodes) #estimate conversion probabilities

#print("True P Matrix: ", prob_matrix[2,:])#prob_matrix[:,2])
#print("Estimated P Matrix: ", estimated_prob)